In [ ]:
from sklearn.metrics import classification_report, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
import importlib.util
from pathlib import Path
import numpy as np


base_path = Path(os.getcwd())
spec = importlib.util.spec_from_file_location("train_xgb", str(base_path  / "../scripts/train_xgb.py"))
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

y_test = module.y_test
y_proba = module.y_proba
y_pred = module.y_pred
X_test = module.X_test

# 1. ROC curve and optimal threshold
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

neg/pos = 18691/5309, scale_pos_weight=3.52
Fitting 3 folds for each of 256 candidates, totalling 768 fits


/Users/olha/explainable-ai/env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:16:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
/Users/olha/explainable-ai/env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:16:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
/Users/olha/explainable-ai/env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:16:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()
/Users/olha/explainable-ai/env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [14:16:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790:


Best parameters: {'alpha': 1, 'colsample_bytree': 0.7, 'gamma': 0, 'lambda': 2, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 1000, 'subsample': 0.8}
Best CV ROC AUC: 0.7841887968617288
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4673
           1       0.66      0.36      0.47      1327

    accuracy                           0.82      6000
   macro avg       0.75      0.65      0.68      6000
weighted avg       0.80      0.82      0.80      6000

ROC AUC: 0.7805954810064262
Confusion matrix:
 [[4431  242]
 [ 850  477]]

=== Threshold: 0.3 ===
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      4673
           1       0.54      0.53      0.54      1327

    accuracy                           0.80      6000
   macro avg       0.71      0.70      0.70      6000
weighted avg       0.80      0.80      0.80      6000


=== Threshold: 0.4 ===
              precision    r

In [ ]:
# 2. Test different thresholds
for thresh in [0.3, 0.4, 0.5]:
    y_pred_custom = (y_proba >= thresh).astype(int)
    print(f"\n=== Threshold: {thresh} ===")
    print(classification_report(y_test, y_pred_custom))

# 3. False negative analysis
fn_mask = (y_test == 1) & (y_pred == 0)
fn_cases = X_test[fn_mask]
print(f"\nFalse Negatives: {fn_mask.sum()}")
print(fn_cases.describe())

# ================================================
# Summary:
# ================================================
# Key Characteristics of Missed Defaults:
# - Avarage credit limit: $155, 786 (relatively high)
# - Payment status (PAY_0-6): Mostly 0 or -1 (paid on time/early)
# - Bill amounts: Moderate (~ $40k average)
# - Payment amounts: Low ($3k avarege) relative to bills


# The model are missing people who:
# Have good credit limits (seem trustworthy)
# Show recent on-time payments (PAY variable near 0)
# But are making minimal payments relative to their balance


=== Threshold: 0.3 ===
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      4673
           1       0.54      0.53      0.54      1327

    accuracy                           0.80      6000
   macro avg       0.71      0.70      0.70      6000
weighted avg       0.80      0.80      0.80      6000


=== Threshold: 0.4 ===
              precision    recall  f1-score   support

           0       0.85      0.93      0.89      4673
           1       0.62      0.43      0.51      1327

    accuracy                           0.82      6000
   macro avg       0.74      0.68      0.70      6000
weighted avg       0.80      0.82      0.80      6000


=== Threshold: 0.5 ===
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4673
           1       0.66      0.36      0.47      1327

    accuracy                           0.82      6000
   macro avg       0.75      0.65      0.68      6000
we